<a href="https://colab.research.google.com/github/pkonowrocki/Multiple-pixels-attack/blob/master/MultiplePixelsAttack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

In [0]:
#install required files
!pip3 install torch torchvision
!pip3 install matplotlib

file_id = '1Iz4CTYuTT-_eS1HrIU410VkeomP5dl0R'

#authentication
from google.colab import auth
auth.authenticate_user()

#build Gdrive service
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

try:
  import io
  from googleapiclient.http import MediaIoBaseDownload

  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    _, done = downloader.next_chunk()
  downloaded.seek(0)
  print('Downloaded file')
  import pickle
  dataset = pickle.load(downloaded)
  assert type(dataset)!="<class 'dict'>", "Something went wrong with downloading file"
  print("Dataset ready")
  
  from urllib.request import urlopen
  idx2label = pickle.load(urlopen('https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl') )
  print("Labels ready")

except:
  print("Something wrong with file")
  print("Trying to download the file and upload it to a Gdrive")
  #downloading dataset
  !curl http://ml.cs.tsinghua.edu.cn/~chenxi/dataset/val224_compressed.pkl --output val224_compressed.pkl
    
  #mounting Gdrive 
  from google.colab import drive
  drive.mount('/content/gdrive')
  #upload dataset to Gdrive
  from googleapiclient.http import MediaFileUpload
  file_metadata = {
    'name': 'val224_compressed'
  }
  media = MediaFileUpload('val224_compressed.pkl',
                        resumable=True)
  created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
  print('File ID: {}'.format(created.get('id')))

Downloaded file
Dataset ready
Labels ready


In [0]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print(device)

import torchvision.models as models
net = models.alexnet(pretrained=True).to(device)
net.eval()

print("Network ready")

cuda:0


Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth
100%|██████████| 244418560/244418560 [00:03<00:00, 70087779.05it/s] 


Network ready


In [0]:
#find positive examples
import numpy as np
import cv2
import torch
import torch.nn as nn

def _str2img(str_b):
    return cv2.imdecode(np.frombuffer(str_b, np.uint8), cv2.IMREAD_COLOR)


mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

print("Got "+str(len(dataset["target"])) +" samples")
m = nn.Softmax()
positive = []

for i in range(len(dataset["target"])):
  image = _str2img(dataset["data"][i])
  image = torch.from_numpy(image).view(1, 3, 224, 224).to(device=device, dtype=torch.float)
  image.div_(255.)
  image[:, 0, :, :].sub_(mean[0]).div_(std[0])
  image[:, 1, :, :].sub_(mean[1]).div_(std[1])
  image[:, 2, :, :].sub_(mean[2]).div_(std[2])
  target = dataset["target"][i]
  output = net(image)
  output = m(output)
  values, indices = torch.max(output, 1)
  if(target==indices.item()):
    print("Image #"+str(i)+"\tprobability: "+str(values.item()*100))
    positive.append((i, values.item()*100))

print(positive)

Got 50000 samples


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Image #125	probability: 70.38662433624268
Image #230	probability: 29.68287467956543
Image #1121	probability: 34.66236591339111
Image #1204	probability: 80.14419674873352
Image #1293	probability: 99.26235675811768
Image #1734	probability: 22.51041829586029
Image #4460	probability: 30.719149112701416
Image #4914	probability: 22.840631008148193
Image #5251	probability: 31.67714774608612
Image #5763	probability: 10.272357612848282
Image #6234	probability: 20.63409835100174
Image #6729	probability: 77.3688018321991
Image #6906	probability: 24.49241131544113
Image #7640	probability: 20.38220465183258
Image #8581	probability: 88.46515417098999
Image #9537	probability: 35.77677011489868
Image #9614	probability: 59.678393602371216
Image #10247	probability: 50.593119859695435
Image #11610	probability: 35.52859425544739
Image #11749	probability: 75.22026896476746
Image #12299	probability: 57.825249433517456
Image #12488	probability: 98.24746251106262
Image #13385	probability: 26.576286554336548
I

In [0]:
positive = [(125, 70.38662433624268), (230, 29.68287467956543), (1121, 34.66236591339111), (1204, 80.14419674873352), (1293, 99.26235675811768), (1734, 22.51041829586029), (4460, 30.719149112701416), (4914, 22.840631008148193), (5251, 31.67714774608612), (5763, 10.272357612848282), (6234, 20.63409835100174), (6729, 77.3688018321991), (6906, 24.49241131544113), (7640, 20.38220465183258), (8581, 88.46515417098999), (9537, 35.77677011489868), (9614, 59.678393602371216), (10247, 50.593119859695435), (11610, 35.52859425544739), (11749, 75.22026896476746), (12299, 57.825249433517456), (12488, 98.24746251106262), (13385, 26.576286554336548), (13627, 41.33410155773163), (13661, 21.732445061206818), (13734, 39.57911729812622), (14359, 14.517764747142792), (15384, 83.67582559585571), (15639, 11.083117872476578), (18307, 8.994264155626297), (19241, 68.18674802780151), (19474, 89.1351580619812), (19753, 72.94465899467468), (20060, 69.57603096961975), (20698, 78.03838849067688), (21335, 56.468266248703), (21461, 77.93958783149719), (22785, 65.87823033332825), (23355, 35.305508971214294), (25701, 29.582718014717102), (26628, 43.83417367935181), (26703, 8.497815579175949), (26822, 37.64067590236664), (27731, 29.25107181072235), (29598, 98.43541383743286), (29876, 66.34024977684021), (31332, 34.71480906009674), (31924, 33.64042639732361), (31969, 76.82187557220459), (32663, 48.51921200752258), (33350, 98.14919829368591), (35491, 67.31398105621338), (39049, 64.29259181022644), (39705, 86.5737795829773), (40235, 35.74826121330261), (40244, 48.99843633174896), (42084, 53.020232915878296), (42275, 13.818688690662384), (42591, 52.02655792236328), (42972, 44.17412579059601), (43095, 71.31219506263733), (44495, 6.340372562408447), (45326, 23.5866978764534), (46550, 72.07267880439758), (47259, 26.392918825149536), (48606, 85.98803877830505), (48816, 49.71528351306915), (48953, 52.26907134056091), (49117, 18.790780007839203), (49648, 34.377798438072205), (49709, 56.61540627479553)]
print(len(positive))

71


# Implementation from paper

In [0]:
import numpy as np

PIXELS_COUNT = 5


class Individual:

    def __init__(self, code):
        self.code = code
        self.obj_fun_value = None

    @staticmethod
    def from_params(x, y, distortion):
        code = np.zeros((PIXELS_COUNT, 5), dtype=float)
        code[:, 0] = x
        code[:, 1] = y
        code[:, 2:5] = distortion
        return Individual(code)

    def get_mutant(self, population, f_param):
        r1, r2, r3 = [ind.code for ind in np.random.choice(population, 3, replace=False)]
        mutant_code = r1 + f_param * (r2 - r3)
        return Individual(mutant_code)

    def crossover_with(self, mutant, c_p, c_rgb):
        crossover_result = np.zeros((PIXELS_COUNT, 5), dtype=float)
        if np.random.rand() < c_p:
            crossover_result[:, 0:2] = self.code[:, 0:2]
        else:
            crossover_result[:, 0:2] = mutant.code[:, 0:2]
        if np.random.rand() < c_rgb:
            crossover_result[:, 2:5] = self.code[:, 2:5]
        else:
            crossover_result[:, 2:5] = mutant.code[:, 2:5]
        return Individual(crossover_result)

    def _crossover(self, first, second, c_r):
        return self._binomial_crossover(
            first,
            self._expotential_crossover(first, second),
            c_r
        )

    def _expotential_crossover(self, first, second):
        result = second
        start = np.random.randint(0, PIXELS_COUNT)
        end = np.random.randint(start, PIXELS_COUNT)
        result[start: end + 1, :] = first[start: end + 1, :]
        return result

    def _binomial_crossover(self, first, second, c_r):
        result = second
        for i in range(PIXELS_COUNT):
            if np.random.rand() < c_r:
                result[i, :] = first[i, :]
        return result
    
    def show_image(self, num_from_dataset):
        import matplotlib.pyplot as plt
        im = np.array(_str2img(dataset["data"][num_from_dataset]), dtype='int32')
        for i in range(self.code.shape[0]):
            im[int(self.code[(i,0)]), int(self.code[(i,1)])] += np.int32(self.code[i,2:5])  
        plt.title(idx2label[dataset["target"][num_from_dataset]])
        plt.imshow(im)
        plt.show()
        
print("Individual loaded")

Individual loaded


In [0]:
import numpy as np
from itertools import product
#from individual import Individual

PIXELS_COUNT = 5


class Population:
    """
    Class representing population of individuals

    """

    def __init__(self, iteration, cardinality, image_size, c_p, c_rgb, f_param, individuals=None):
        self.iteration = iteration
        self.cardinality = cardinality
        self.image_size = image_size
        self.c_p = c_p
        self.c_rgb = c_rgb
        self.f_param = f_param

        if individuals == None:
            self.individuals = self._initialize_individuals(
                cardinality, image_size)
        else:
            self.individuals = individuals

    def _initialize_individuals(self, cardinality, image_size):
        individuals = []
        height, width = image_size
        possible_positions = np.array(
            list(product(range(height), range(width))))
        positions = possible_positions[np.random.randint(0,
                                                         height * width, cardinality * PIXELS_COUNT)]

        for i in range(cardinality):
            code = np.zeros((PIXELS_COUNT, 5))
            code[:, 2:5] = np.random.normal(128, 127, (PIXELS_COUNT, 3))
            code[:, 0:2] = positions[i * PIXELS_COUNT: i *
                                     PIXELS_COUNT + PIXELS_COUNT]
            individuals.append(self._fixed_individual(Individual(code)))
            i += 1

        return individuals

    def generate_next(self, objective_function):
        new_individuals = []
        for i in range(self.cardinality):
            parent = self.individuals[i]
            mutant = parent.get_mutant(
                self.individuals, self.f_param)
            mutant = self._fixed_individual(mutant)
            descendant = parent.crossover_with(
                mutant, self.c_p, self.c_rgb)
            descendant.obj_fun_value = objective_function(descendant)
            
            if parent.obj_fun_value == None:
                parent.obj_fun_value = objective_function(parent)
              
            if descendant.obj_fun_value >= parent.obj_fun_value:
                new_individuals.append(descendant)
            else:
                new_individuals.append(parent)
            i += 1
        return Population(self.iteration + 1, self.cardinality, self.image_size, self.c_p, self.c_rgb, self.f_param, new_individuals)
      
    def get_best_individual(self):
      return self.individuals[np.argmax([ind.obj_fun_value for ind in self.individuals])]
    
    def _fixed_individual(self, individual):
      height, width = self.image_size
      new_code = np.zeros((PIXELS_COUNT, 5))
      new_code[:,0] = np.floor(individual.code[:,0] % height)
      new_code[:,1] = np.floor(individual.code[:,1] % width)
      new_code[:,2:5] = individual.code[:,2:5] % 256
      return Individual(new_code)

In [0]:
import torch.nn as nn
import numpy as np
import cv2
import torch


#softmax layer to normalize output
m = nn.Softmax()

#mean and std of input normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

#convert str to image
def _str2img(str_b):
    return cv2.imdecode(np.frombuffer(str_b, np.uint8), cv2.IMREAD_COLOR)

def find_probabilities(individual, num_from_dataset):
    im = np.array(_str2img(dataset["data"][num_from_dataset]), dtype='float64')
    for i in range(individual.code.shape[0]):
        im[int(individual.code[(i,0)]), int(individual.code[(i,1)])] = individual.code[i,2:5]
    im = torch.from_numpy(im).view(1, 3, 224, 224).to(device=device, dtype=torch.float)
    im.div_(255.)
    im[:, 0, :, :].sub_(mean[0]).div_(std[0])
    im[:, 1, :, :].sub_(mean[1]).div_(std[1])
    im[:, 2, :, :].sub_(mean[2]).div_(std[2])
    output = net(im)
    return m(output)
  
def find_best_class(individual, num_from_dataset):
    output = find_probabilities(individual, num_from_dataset)
    values, indices = torch.max(output, 1)
    return values.item(), indices.item()
  
def find_second_best_class(individual, num_from_dataset):
    output = find_probabilities(individual, num_from_dataset)
    values, indices = torch.sort(output, 1, True)
    return values[0][1].item(), indices[0][1].item()
  
def fitness_function(individual, num_from_dataset):
    output = find_probabilities(individual, num_from_dataset)
    return output.cpu().detach().numpy()[0, dataset["target"][num_from_dataset]]
  
print("Fitness function loaded")

Fitness function loaded


In [0]:
def differential_evolution(iter_count, population_cardinality, image_size, num_from_dataset, c_p, c_rgb, f_param, obj_function):
  population = Population(0, population_cardinality, image_size, c_p, c_rgb, f_param)
  
  for i in range(iter_count):
    population = population.generate_next(obj_function)
    
  return population.get_best_individual()

In [0]:
def calc_average_distortion(individual, num_from_dataset):
  im = np.array(_str2img(dataset["data"][num_from_dataset]), dtype='float64')
  distortion = 0
  n = individual.shape[0]
  for i in range(n):
    distortion += sum(abs(im[int(individual[i, 0]), int(individual[i, 1])] - individual[i, 2:5]))
  return distortion / n


def test_params(c_p, c_rgb, f_param):
  success_count = 0
  misclasified_probability = 0
  distortion = 0
  for num in [item[0] for item in sorted(positive, key=lambda tup: tup[1])]:
    best_individual = differential_evolution(100, 20, (224, 224), num, c_p, c_rgb, f_param, lambda individual: 1 - fitness_function(individual, num))
    probability, label_index = find_best_class(best_individual, num)
    if dataset["target"][num] != label_index:
      success_count += 1
      misclasified_probability += probability
      distortion += calc_average_distortion(best_individual.code, num)
  return success_count / len(positive), misclasified_probability / success_count, distortion / success_count


for c_p in range(1,10, 2):
  for c_rgb in range(1,10, 2):
    for f_param in range(1,10, 2):
      success_rate, confidence, average_distortion = test_params(c_p / 10, c_rgb / 10, f_param / 10)
      print(c_p / 10, c_rgb /10, f_param / 10, success_rate, confidence, average_distortion)
      
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.1 0.9 0.1 0.1267605633802817 0.27624886275993454 333.4110609519519
0.1 0.9 0.3 0.11267605633802817 0.2796528795734048 334.7674208603127
0.1 0.9 0.5 0.07042253521126761 0.33598680049180984 338.9497603868469
0.1 0.9 0.7 0.056338028169014086 0.3571000397205353 340.8385174572572
0.1 0.9 0.9 0.09859154929577464 0.2201451967869486 310.75590326751353


# New implementation

In [0]:
import numpy as np
import random

PIXELS_COUNT = 25

class Individual:

    def __init__(self, code, num_from_dataset):
        self.num_from_dataset = num_from_dataset
        self.image = np.array(_str2img(dataset["data"][num_from_dataset]), dtype='int32')
        self.code = code
        self.fitness = None
        for (k,v) in code.items():
            self.code[k] = (self.image[k]+self.code[k])%256 - self.image[k]
        
    @staticmethod
    def from_params(x, y, d, distortion, num_from_dataset):
        code = {}
        code[(x, y, d)] = distortion
        return Individual(code, num_from_dataset)
    
    def __add__(self, o):
        result = dict(self.code)
        for k in o.code.keys():
            if k in result.keys():
                result[k] += o.code[k]
            else:
                result[k] = o.code[k]
        
        return Individual(result, self.num_from_dataset)

    def __sub__(self, o):
        result = {}
        result = dict(self.code)
        for k in o.code.keys():
            if k in result.keys():
                result[k] -= o.code[k]
            else:
                result[k] = -o.code[k]
        return Individual(result, self.num_from_dataset)

    def __rmul__(self, o):
        result = {}
        for k in self.code.keys():
            result[k] = o*self.code[k]
        return Individual(result, self.num_from_dataset)
    
    def __mul__(self, o):
        result = {}
        for k in o.code.keys():
            result[k] = self*o.code[k]
        return Individual(result, self.num_from_dataset)
    
    def get_mutant(self, population, f_param):
        r1, r2, r3 = np.random.choice(population, 3, replace=False)
        mutant = r1 + f_param * (r2 - r3)
        return mutant

    def crossover_with(self, mutant, c_r):
        result = {}
        shuffled_keys = list(self.code.keys())
        
        shuffled_mutant_keys = list(mutant.code.keys())
        random.shuffle(shuffled_mutant_keys)
        
        for _ in range(PIXELS_COUNT):
            if c_r > np.random.rand():
                
                if shuffled_mutant_keys:
                    new_key = shuffled_mutant_keys.pop()
                    if new_key not in result.keys():
                        result[new_key] = mutant.code[new_key]
                    else:
                        result[new_key] = mutant.code[new_key] + result[new_key]
            else:
                if shuffled_keys:
                    new_key = shuffled_keys.pop()
                    if new_key not in result.keys():
                        result[new_key] = self.code[new_key]
                    else:
                        result[new_key] = self.code[new_key] + result[new_key]
        return Individual(result, self.num_from_dataset)
    
    def show_image(self):
        import matplotlib.pyplot as plt
        im = self.image
        plt.title(idx2label[dataset["target"][self.num_from_dataset]])
        plt.imshow(im)
        plt.show()
        
    def show_output(self):
        import matplotlib.pyplot as plt
        im = np.array(self.image, dtype='float64')
        for (k,v) in self.code.items():
            self.code[k] = (self.image[k]+self.code[k])%256 - self.image[k]    
        img = torch.from_numpy(im).view(1, 3, 224, 224).to(device=device, dtype=torch.float)
        img.div_(255.)
        img[:, 0, :, :].sub_(mean[0]).div_(std[0])
        img[:, 1, :, :].sub_(mean[1]).div_(std[1])
        img[:, 2, :, :].sub_(mean[2]).div_(std[2])
        output = net(img)
        output = m(output)
        value, indice = torch.max(output, 1)
        plt.title(idx2label[indice.item()])
        plt.imshow(im.astype(int))
        plt.show()
        
print("Individual loaded")

Individual loaded


In [0]:
import numpy as np
from itertools import product


class Population:
    """
    Class representing population of individuals

    """

    def __init__(self, iteration, cardinality, image_size, c_r, f_param, num_from_dataset, individuals=None):
        self.iteration = iteration
        self.cardinality = cardinality
        self.image_size = image_size
        self.c_r = c_r
        self.f_param = f_param
        self.num_from_dataset = num_from_dataset

        if individuals == None:
            self.individuals = self._initialize_individuals(
                cardinality, image_size, num_from_dataset)
        else:
            self.individuals = individuals

    def _initialize_individuals(self, cardinality, image_size, num_from_dataset):
        individuals = []
        height, width = image_size
        
        for _ in range(cardinality-1):
            code = {}
            for _ in range(PIXELS_COUNT):
                x = np.random.randint(width)
                y = np.random.randint(height)
                d = np.random.randint(3)
                disortion = 15*np.random.randn()
                code[(x, y, d)] = disortion
            individuals.append(Individual(code, num_from_dataset))
        individuals.append(Individual({}, num_from_dataset))
        return individuals

    def generate_next(self, objective_function):
        new_individuals = []
        for i in range(self.cardinality):
            parent = self.individuals[i]
            mutant = parent.get_mutant(
                self.individuals, self.f_param)
            #descendant = parent.crossover_with(
            #    mutant, self.c_r)
            descendant = mutant
            descendant.fitness = objective_function(descendant)
            
            if parent.fitness == None:
                parent.fitness = objective_function(parent)
              
            if descendant.fitness < parent.fitness:
                new_individuals.append(descendant)
            else:
                new_individuals.append(parent)
            i += 1
        return Population(self.iteration + 1, self.cardinality, self.image_size, self.c_r, self.f_param, self.num_from_dataset, new_individuals)
      
    def get_best_individual(self):
      return self.individuals[np.argmax([ind.fitness for ind in self.individuals])]
    
print("Population loaded")

Population loaded


In [0]:
import torch.nn as nn
import numpy as np
import cv2
import torch


#softmax layer to normalize output
m = nn.Softmax()

#mean and std of input normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

#convert str to image
def _str2img(str_b):
    return cv2.imdecode(np.frombuffer(str_b, np.uint8), cv2.IMREAD_COLOR)


def fitness_function(individual):
    im = individual.image #np.array(_str2img(dataset["data"][num_from_dataset]), dtype='float64')
    for (k,v) in individual.code.items():
        im[k] = im[k] + v          
    im = torch.from_numpy(im).view(1, 3, 224, 224).to(device=device, dtype=torch.float)
    im.div_(255.)
    im[:, 0, :, :].sub_(mean[0]).div_(std[0])
    im[:, 1, :, :].sub_(mean[1]).div_(std[1])
    im[:, 2, :, :].sub_(mean[2]).div_(std[2])
    output = net(im)
    output = m(output)
    return output.cpu().detach().numpy()[0, dataset["target"][individual.num_from_dataset]]
  
print("Fitness function loaded")

Fitness function loaded


In [0]:
import csv
import time

every = 100

def differential_evolution(iter_count, population_cardinality, image_size, num_from_dataset, c_r, f_param):
  population = Population(0, population_cardinality, image_size, c_r, f_param, num_from_dataset)
  objective_function = lambda individual: fitness_function(individual)
  
  writer = csv.writer(open("OWN1",'w+'))
  t_zero = time.time()
  for i in range(iter_count):
    
    offspring = population.generate_next(objective_function)
    writer.writerow([i, population.get_best_individual().fitness])
    if i%every==0:
        print("Iteration: " + str(i) +"\tbest fitness value: "+str(population.get_best_individual().fitness) + "\tremaining time "+str((time.time()-t_zero)*(iter_count/(i+1)-1)/60)+" min")
        #print(population.get_best_individual().code)
    population = offspring
    
  return population.get_best_individual()

print("Differential evolution algorithm loaded")

Differential evolution algorithm loaded


In [0]:
#image_num = 8411 # >90%
image_num = 18307	 



fooled = differential_evolution(10000, 50, (224,224), image_num, 0.9, 0.5)
fooled.show_image()
fooled.show_output()
print(fooled.code)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Iteration: 0	best fitness value: 0.09212413	remaining time 108.61665526628494 min
Iteration: 100	best fitness value: 0.058484435	remaining time 161.2183360656496 min
Iteration: 200	best fitness value: 0.05318091	remaining time 161.48351525758235 min
Iteration: 300	best fitness value: 0.05000746	remaining time 161.16841895425836 min
Iteration: 400	best fitness value: 0.048260976	remaining time 159.83822690192008 min
Iteration: 500	best fitness value: 0.046882488	remaining time 158.3332632941718 min
Iteration: 600	best fitness value: 0.046306614	remaining time 156.70666702061445 min
Iteration: 700	best fitness value: 0.045574415	remaining time 154.89477081422405 min
Iteration: 800	best fitness value: 0.045031432	remaining time 153.13659401689526 min
Iteration: 900	best fitness value: 0.044386692	remaining time 151.46467800083488 min
Iteration: 1000	best fitness value: 0.04378202	remaining time 149.79264697907092 min
Iteration: 1100	best fitness value: 0.043277528	remaining time 148.10828

Sources: [Dataset](https://github.com/aaron-xichen/pytorch-playground) [Neural network model](https://pytorch.org/docs/stable/torchvision/models.html)